# Initialisation

In [719]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#%matplotlib tk #pour Linux

In [720]:
import scipy.ndimage as ndimage
from sklearn.cluster import DBSCAN

# Importer fichier

In [721]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_1um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [722]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


71

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [723]:
frame_index = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    print(frame_index)

1


# Traitement d'image

In [724]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Sélection du point à tracker

In [725]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, origin='lower', cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze_x = crop_sze_y = 50

Please click on the point you want to select.
Selected point: (915.279220779221, 698.461038961039)


In [726]:
def crop(img, x, y, crop_size_x, crop_size_y):
    x_start = int(x - crop_size_x // 2)
    x_end = int(x + crop_size_x // 2)
    y_start = int(y - crop_size_y // 2)
    y_end = int(y + crop_size_y // 2)

    return img[y_start:y_end, x_start:x_end]

# Fit gaussien sur le point sélectionné

In [727]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [728]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [729]:
def localisateur_gaussien(intensity_grid, maxi):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))

    x_position = result.params['x0'].value + maxi[0] - crop_sze_x/2
    y_position = result.params['y0'].value + maxi[1] - crop_sze_y/2

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [730]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Passe au prochain frame

In [731]:
def next_frame(frame_i):
    frame_i += 1
    tiff.seek(frame_i)
    original_image = np.array(tiff)
    
    while np.sum(original_image) == 0:
        frame_i += 1
        tiff.seek(frame_i)
        original_image = np.array(tiff)

    return [frame_i, original_image]

Débogueur

In [732]:
def visionneur(frame):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title('Grille Zoomée avec Position')
    plt.colorbar()  
    plt.show()

# Identifier la particule et donner sa nouvelle position

In [733]:
def empreinte_digitale(image, threshold=12, eps=3, min_samples=1):
    # 1. Observer chaque particule approximativement
    #neighborhood_size = 3  
    #local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)

    # 2. Déterminer l'empreinte digitale de notre particule
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            size = cluster_points.shape[0]
            luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])
    
    seuil=np.max(image)*0.4
    print(seuil)
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    plt.legend()
    plt.colorbar()  
    plt.show()

    return size, luminosity, seuil

In [734]:
def detect_nbre_particles(image, threshold, eps=3, min_samples=1):
    # 1. Appliquer un filtre de voisinage pour détecter les maxima locaux
    neighborhood_size = 3  # Taille du voisinage pour détecter les maxima locaux
    local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    # 2. Comparer l'image originale et les maxima locaux pour identifier les vrais maxima
    maxima = (image == local_max) & (image > threshold)

    # 3. Extraire les coordonnées des maxima (particules)
    coordinates = np.column_stack(np.where(maxima))  # Extraire les indices des pixels maximaux
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        visionneur(image)

        return [], image

    # 4. Appliquer DBSCAN pour regrouper les particules proches (cluster les maxima détectés)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)

    #5. Déterminer le nombre de particules
    unique_labels = np.unique(labels)
    nb_particules = len(unique_labels)
        
    # 6. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    
    positions=[]
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')
#        if nb_particules > 1:
        positions.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))
        

    plt.colorbar()  
    plt.legend()
    plt.show()

    # Retourner les coordonnées des particules filtrées
    return nb_particules, positions

In [735]:
def identificateur_particles(image, ref_size, ref_luminosity, threshold=25, eps=3, min_samples=1):
    # 1. Obtenir le plus d'info sur la particule
    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Mauvais re-crop empêche identification de la particule.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)

    # 2. Parcourir chaque particule et identifier laquelle est celle qu'on suit
    distances = []
    position_particules = []

    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            
            size = cluster_points.shape[0]
            luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])
            size_distance = abs(size - ref_size)  
            luminosity_distance = abs(luminosity - ref_luminosity)  
            combined_distance = size_distance + luminosity_distance
            distances.append(combined_distance)
            position_particules.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))

    ecart = np.min(distances)
    position_best_particle = position_particules[np.argmin(distances)]  
    print(f'ecrat: {ecart}')
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    plt.scatter(position_best_particle[0], position_best_particle[1], color='r', label=f'Particule sélectionnée')
    plt.legend()
    plt.colorbar()  
    plt.show()

    return position_best_particle, ecart

In [736]:
def particle_tracker(image, positions, frame_index, taille_initiale, luminosité_initiale, seuil):
    # 1. Ce qu'on voit près de l'ancienne position
    image = denoise(image)
    #print(positions[frame_index][0],positions[frame_index][1])

    cropped_img = crop(image, positions[frame_index][0], positions[frame_index][1], crop_sze_x, crop_sze_y)
    
    # 2. Déterminer combien il y a de particules et laquelle est la notre
    # Utiliser une reconnaissance de particules et si plus qu'une, alors clic pour choisir
    nb_part, coordonnées = detect_nbre_particles(cropped_img, seuil)
    print(f'coordonnées des {nb_part} particules:{coordonnées}')

    if (nb_part == 0) or (nb_part == []):
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(denoise(img), origin='lower', cmap='gray') 
        plt.title(f"Frame {frame_index}: Select a point")

        # Ajouter une croix rouge à la position donnée
        cx, cy = positions[frame_index-1]
        ax.plot(cx, cy, 'rx', markersize=10) 
        
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        nouveau_x, nouveau_y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({x}, {y})")
        
        # Fermer l'affichage après sélection
        plt.close()
    elif nb_part > 1:
        imperfections, emplacements = [] , []
        for num_particule in range(nb_part):
            x_identifier=positions[frame_index][0] - crop_sze_x // 2 + coordonnées[num_particule][0]
            y_identifier=positions[frame_index][1] - crop_sze_y // 2 + coordonnées[num_particule][1]
            crop_pour_identifier = crop(image, x_identifier, y_identifier, crop_sze_x, crop_sze_y)
            emplacement, ecart = identificateur_particles(crop_pour_identifier, taille_initiale, luminosité_initiale, seuil)
            imperfections.append(ecart)
            emplacements.append(emplacement)
        print(imperfections)
        print(np.argmin(imperfections))
        qualité = np.min(imperfections)
        max_coords = emplacements[np.argmin(imperfections)]  
        print(qualité)
        print(max_coords)
        nouveau_x = x_identifier - crop_sze_x // 2 + max_coords[0]
        nouveau_y = y_identifier - crop_sze_y // 2 + max_coords[1]
    else:
        cropped_img = np.array(cropped_img)
        max_index = np.argmax(cropped_img)
        max_coords = np.unravel_index(max_index, cropped_img.shape)
    #print(F'coordonnées du max dans le premier crop: ({max_coords[1]},{max_coords[0]})')
    #if frame_index>1:
    #    visionneur(cropped_img)

        nouveau_x = positions[frame_index][0] - crop_sze_x // 2 + max_coords[1] #Oui c'est inversé à cause du unravel juste avant
        nouveau_y = positions[frame_index][1] - crop_sze_y // 2 + max_coords[0]

    #print(f'(x,y) central du second crop: {nouveau_x, nouveau_y}')
    
    # 3. Crop atour de notre particule et fit dessus
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x, crop_sze_y)    # Re-crop autour d'une seule particule

    #if frame_index>1:
    #    visionneur(second_crop)


    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])
    #print(f'écart-type: ({result_fit[1]}, {result_fit[2]})')

    x_new, y_new = result_fit[0][0], result_fit[0][1]
    #print(f'Positions finales de l index {frame_index}: ({x_new}, {y_new}) ')
    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [737]:
position_list = [(x,y), (x,y)]
sigma_list = []
crop_frames = []
big_frames = []
taille_initiale, luminosité_initiale, seuil = empreinte_digitale(crop(img,x,y,crop_sze_x,crop_sze_y))

for _ in range(27 - 1):
    print(f'frame index: {frame_index} et len(positions): {len(position_list)}')
    data = particle_tracker(img, position_list, frame_index, taille_initiale, luminosité_initiale, seuil)
    position_list.append(data[2])
    sigma_list.append(data[3])
    frame_index, img = next_frame(frame_index)
    big_frames.append(img)
    crop_frames.append(data[1])

21.200000000000003


C:\Users\maxim\AppData\Local\Temp\ipykernel_12620\3821038158.py:28: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


frame index: 1 et len(positions): 2
coordonnées des 1 particules:[(25.666666666666668, 14.5)]
frame index: 2 et len(positions): 3
coordonnées des 1 particules:[(26.6, 24.2)]
frame index: 3 et len(positions): 4
coordonnées des 1 particules:[(20.666666666666668, 28.333333333333332)]
frame index: 4 et len(positions): 5
coordonnées des 1 particules:[(23.5, 31.75)]
frame index: 5 et len(positions): 6
coordonnées des 1 particules:[(27.6, 25.2)]
frame index: 6 et len(positions): 7
coordonnées des 1 particules:[(27.666666666666668, 22.666666666666668)]
frame index: 7 et len(positions): 8
coordonnées des 1 particules:[(33.0, 35.75)]
frame index: 8 et len(positions): 9
coordonnées des 1 particules:[(27.0, 23.666666666666668)]
frame index: 9 et len(positions): 10
coordonnées des 1 particules:[(26.0, 21.5)]
frame index: 10 et len(positions): 11
coordonnées des 1 particules:[(27.333333333333332, 29.166666666666668)]
frame index: 11 et len(positions): 12
coordonnées des 1 particules:[(29.0, 32.5)]
f

# Résultats

In [738]:
position_list

[(915.279220779221, 698.461038961039),
 (915.279220779221, 698.461038961039),
 (916.4793395563825, 689.0194419145745),
 (918.289445521969, 687.8621494783184),
 (914.6866810487812, 689.4288620303947),
 (913.0007733014323, 695.6606211721459),
 (915.7807161684442, 695.3193877898689),
 (918.0468401061239, 693.3170024893979),
 (926.1833825413155, 703.2680006734216),
 (928.4442789446889, 702.7333118056706),
 (929.0890345310461, 698.3761103134564),
 (931.7335045281454, 701.8726180355416),
 (936.2171623402277, 709.315988237023),
 (939.1438630449828, 702.4858194605986),
 (942.4150855766092, 700.9371710829006),
 (946.3042972910837, 702.0349056135238),
 (942.9058330425007, 705.7087327448813),
 (942.7278453247119, 712.2231084649534),
 (958.7094659193457, 738.2156630098333),
 (919.7194998686869, 732.8972122592158),
 (941.0143688628079, 717.934666654229),
 (945.187951874992, 723.3095655077065),
 (942.6882350320035, 720.5921119108676),
 (943.3717385933688, 725.6004240916035),
 (950.9168026177848, 730

In [739]:
sigma_list

[(4.174531674129262, 5.7332330066249),
 (2.9432521656223667, 3.8785565438775857),
 (3.2324558260714, 4.187085110613818),
 (3.381480568300901, 3.69359523223278),
 (3.1113192814615873, 3.8851385216892163),
 (3.5147974913324505, 4.003050123307734),
 (3.084769292358209, 4.094801379180481),
 (3.1594010712832308, 4.3444618225026055),
 (3.089703455145231, 3.792325847382228),
 (3.1170689263596048, 3.9902644452382274),
 (2.653296873342459, 3.5666501469221443),
 (2.836311313390358, 3.93367699925955),
 (2.9893607728887552, 3.9550438662740173),
 (3.101552912630669, 4.232803953914599),
 (2.7306758323185374, 4.0409184032546035),
 (3.4114213041804273, 3.2632296682089064),
 (3.579089643968472, 3.6114492083898075),
 (-6.66365688786784, -89.36241287351933),
 (3.159690301278375, 3.7038589674726117),
 (3.221757821073098, 3.9941011751445026),
 (2.9921474025841075, 3.941128563849096),
 (2.9774454229075986, 4.359089637513783),
 (3.4820190103318525, 4.538538196921804),
 (2.949975328517128, 4.418407064342226),

In [740]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Inverser l'axe Y
plt.gca().invert_yaxis()

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

# Afficher le graphique
plt.show()

## anim toutes les particules

In [741]:
fig, ax = plt.subplots()
img = ax.imshow(big_frames[0], origin='lower', cmap='gray', animated=True)


# Update function
def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
plt.show()

## anim crop

In [742]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], origin='lower', cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()